In [1]:
import altair as alt
import pandas as pd
import altair_viewer
alt.data_transformers.disable_max_rows()
altair_viewer._global_viewer._use_bundled_js = False
alt.data_transformers.enable('data_server')

Measure = ['Quantity','Profit','Discount','Sales']

def filterDate(Dimension,typ): #Date inly

    df[Dimension] = pd.to_datetime(df[Dimension],format='%d/%m/%Y')

    if typ == 'year':
        s = str(Dimension+' year')
        df[s] = df[Dimension].dt.year
        return df[s]
    elif typ == 'month':
        s = str(Dimension+' month')
        df[s] = df[Dimension].dt.month
        return df[s]
    elif typ == 'day':
        s = str(Dimension+' day')
        df[s] = df[Dimension].dt.day
        return df[s]

def rangeScale(Di,Meas):
    fil = Meas[1]
    if (type(Di[0]) == type(['list'])) and (type(Di[1]) == type(['list'])):
        x = str(Di[1][0]+' '+Di[1][1])
        col = str(Di[0][0]+' '+Di[0][1])
    elif type(Di[0]) == type(['list']):
        x = Di[1]
        col = str(Di[0][0]+' '+Di[0][1])
    elif type(Di[1]) == type(['list']):
        x = str(Di[1][0]+' '+Di[1][1])
        col = Di[0]
    else:
        x = Di[1]
        col = Di[0]
    
    tmax = df.groupby([col,x], as_index=False)[Meas[0]].agg(fil).max()[2]
    tmin = df.groupby([col,x], as_index=False)[Meas[0]].agg(fil).min()[2]
    
    if tmin > 0:
        tmin = 0
    elif tmax < 0 :
        tmax = 0
    
    return [tmin,tmax]

def functionRC(row,column):
    lr = []
    lc = []

    for r in row:
        if type(r) == type(['list']):
            s = str(r[1]+'('+r[0]+')')
            lr.append(s)
        else:
            lr.append(r)

    for c in column:
        if type(c) == type(['list']):
            s = str(c[1]+'('+c[0]+')')
            lc.append(s)
        else:
            lc.append(c)
    
    return [lr,lc]

def plotBar(row,column,meas,di,mes): #edit and add dimen in 3 dimen
    print(row,column)
    l = functionRC(row,column)
    lr = l[0]
    lc = l[1]
    
    if len(lr) == 1 and len(lc) == 1:
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=lr[-1]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:
        if mes == 'row':
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lr[-2],lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lr[-2],lr[-1],lc[-1]]
            ).facet(row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 1 and len(lc) == 2:
        if mes == 'row':
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lc[-2],lr[-1],lc[-1]]
            ).facet(column=lc[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 2 and len(lc) == 2:
        if mes == 'row':
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-1],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                tooltip = [lc[-2],lr[-1],lc[-2],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                tooltip = [lc[-2],lr[-1],lc[-2],lc[-1]]
            ).facet(column=lc[-2],row=lr[-2]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c


    elif len(lr) == 1 and len(lc) == 3:
        if mes == 'row':
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-2],
                y=alt.Y(lr[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                color = lc[-1],
                tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
            ).facet(column=lc[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-2],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-1],
                color = lc[-1],
                tooltip = [lc[-3],lc[-2],lc[-1],lr[-1]]
            ).facet(column=lc[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    elif len(lr) == 3 and len(lc) == 1:
        if mes == 'row':
            c = alt.Chart(df).mark_bar().encode(
                x=lc[-1],
                y=alt.Y(lr[-2],scale=alt.Scale(domain=rangeScale(di,meas))),
                color = lr[-1],
                tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
            ).facet(row=lr[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c
        else:
            c = alt.Chart(df).mark_bar().encode(
                x=alt.X(lc[-1],scale=alt.Scale(domain=rangeScale(di,meas))),
                y=lr[-2],
                color = lr[-1],
                tooltip = [lc[-1],lr[-3],lr[-2],lr[-1]]
            ).facet(row=lr[-3]
            ).resolve_scale(x = 'independent',y = 'independent')
            return c

    
        
def plotLine(row,column,mes,Dimen):

    print(row,column)
    if mes == 'column':                 #column is Measure
        fil = column[1]
        Me = column[0]
        if Dimen == 1:
            Di = row[0][0]
            fd = row[0][1]
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q')),
                alt.Y(str(fd+'('+Di+'):T')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        elif Dimen == 2:
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fil+'('+Me+'):Q'),scale=alt.Scale(domain=rangeScale([row[0],row[1]],column))),
                alt.Y(str(row[-1][1]+'('+row[-1][0]+'):T')),
                row = str(row[-2][1]+'('+row[-2][0]+'):T'),
                tooltip = [str(row[-1][1]+'('+row[-1][0]+'):T'),str(fil+'('+Me+')')]
            ).resolve_legend(
                size='independent'
            ).resolve_scale(
                y = 'independent',x = 'independent'
            )
            return ch

    elif mes == 'row':   #row is Measure
        Me = row[0]
        fil = row[1]
        #elif df[c].dtypes == 'datetime64[ns]':
        if Dimen == 1:
            Di = column[0][0]
            fd = column[0][1]
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.X(str(fd+'('+Di+'):T')),
                alt.Y(str(fil+'('+Me+'):Q')),
                tooltip = [str(fd+'('+Di+'):T'),str(fil+'('+Me+'):Q')]
            )
            return ch
        elif Dimen == 2:
            ch = alt.Chart(df).mark_line(point=True).encode(
                alt.Y(str(fil+'('+Me+'):Q'),scale=alt.Scale(domain=rangeScale([column[0],column[1]],row))),
                alt.X(str(column[-1][1]+'('+column[-1][0]+'):T')),
                column = str(column[-2][1]+'('+column[-2][0]+'):T'),
                tooltip = [str(column[-1][1]+'('+column[-1][0]+'):T'),str(fil+'('+Me+')')]
            ).resolve_legend(
                size='independent'
            ).resolve_scale(
                y = 'independent',x = 'independent'
            )
            return ch

def plotPie(row,column,mes):
    if mes == 'row':
        Mes = row[0]
        fil = row[1]
        Di = column[0]
    elif mes == 'column':
        Mes = column[0]
        fil = column[1]
        Di = row[0]

    if type(Di) == type([]):
        s = str(Di[1]+'('+Di[0]+'):T')
    else:
        s = str(Di+':N')

    base = alt.Chart(df).mark_arc().encode(
        theta=alt.Theta(str(fil+'('+Mes+'):Q')), 
        color=alt.Color(s, type="nominal"), 
        tooltip = [s,str(fil+'('+Mes+'):Q')]
    )

    return base

def plotGraph(row,column,tp):

    def checkMeasure(R,C):      #True when row is measure
        for r in R:
            if type(r) == type([]):
                if r[0] in Measure:
                    return True
                else:
                    return False
            else:
                return False

    if tp == 'line':
        if checkMeasure(row,column):    #row is measure
            print('row is measurement')
            chart = []
            for r in row:
                chart.append(plotLine(r,column,'row',len(column)))
            return alt.vconcat(*chart)
        else:                       #column is Measurement
            print('column is measurement')
            chart = []
            for c in column:
                chart.append(plotLine(row,c,'column',len(row)))
            return alt.hconcat(*chart)

    elif tp == 'bar':
        mes = 'col'
        for r in row:
            if type(r) == type(['list']):
                if r[0] in Measure:
                    mes = 'row'

        if mes == 'row':
            chart = []
            if type(row[0]) == type(['list']):
                if row[0][0] not in Measure:
                    l = [*column]
                    l.append(row[0])
                    for r in range(len(row)-1):
                        chart.append(plotBar([row[0],row[r+1]],column,row[r+1],l,mes))
                    return alt.vconcat(*chart)
                else:
                    for r in range(len(row)):
                        chart.append(plotBar([row[r]],column,row[r],[*column],mes))
                    return alt.vconcat(*chart)
            else:
                l = [*column]
                l.append(row[0])
                for r in range(len(row)-1):
                    chart.append(plotBar([row[0],row[r+1]],column,row[r+1],l,mes))
                return alt.vconcat(*chart)
        else:
            chart = []
            if type(column[0]) == type(['list']):
                if column[0][0] not in Measure:
                    for c in range(len(column)-1):
                        chart.append(plotBar(row,[column[0],column[c+1]],column[c+1]))
                    return alt.vconcat(*chart)
                else:
                    for c in range(len(column)):
                        chart.append(plotBar(row,[column[c]],column[c]))
                    return alt.vconcat(*chart)
            else:
                for c in range(len(column)-1):
                    chart.append(plotBar(row,[column[0],column[c+1]],column[c+1]))
                return alt.vconcat(*chart)

    elif tp == 'pie':
        if checkMeasure(row,column):    #row is measure
            print('row is measurement')
            chart = []
            for r in row:
                chart.append(plotPie(r,column,'row'))
            return alt.vconcat(*chart)
        else:
            print('column is measurement')
            chart = []
            for c in column:
                chart.append(plotPie(row,c,'column'))
            return alt.hconcat(*chart)

df = pd.read_csv('Superstore.csv', encoding='windows-1252')

for d in ['Order Date','Ship Date']:
    filterDate(d,'year')
    filterDate(d,'month')
    filterDate(d,'day')

D1 = ['Order Date','month']
D2 = ['Ship Date','year']

D3 = 'Category'
D4 = 'Sub-Category'

D5 = 'Country/Region'
D6 = 'Region'
D7 = 'State'
D8 = 'City'

D9 = 'Ship Mode'
D10 = 'Segment'

M1 = ['Quantity','sum']
M2 = ['Discount','sum']
M3 = ['Profit','sum']
M4 = ['Sales','sum']

row = [D9,M2,M3]
column = [D3,D10]

#column , row = row ,column

#plotBar(row,column,f1)
#plotLine(row,column,f1)
#plotPie(row,column,f1)
plotGraph(row,column,'bar')

['Ship Mode', ['Discount', 'sum']] ['Category', 'Segment']
['Ship Mode', ['Profit', 'sum']] ['Category', 'Segment']


alt.VConcatChart(...)

In [90]:
def plotBar(row,column,mes):
    l = functionRC(row,column)
    lr = l[0]
    lc = l[1]
    
    if len(lr) == 1 and len(lc) == 1:
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=lr[-1]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c
        
    elif len(lr) == 2 and len(lc) == 1:
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=lr[-1]
        ).facet(row=lr[-2]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c

    elif len(lr) == 1 and len(lc) == 2:
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=lr[-1]
        ).facet(column=lc[-2]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c

    elif len(lr) == 2 and len(lc) == 2:
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=lr[-1]
        ).facet(row=lr[-2] , column = lc[-2]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c

    elif len(lr) == 1 and len(lc) == 3:
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-2],
            y=lr[-1],
            color = lc[-1]
        ).facet(column = lc[-3]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c

    elif len(lr) == 3 and len(lc) == 1:
        c = alt.Chart(df).mark_bar().encode(
            x=lc[-1],
            y=lr[-2],
            color = lr[-1]
        ).facet(row=lr[-3]
        ).resolve_scale(x = 'independent',y = 'independent')
        return c

In [69]:
D1 = ['Order Date','month']
D2 = ['Ship Date','year']

D3 = 'Category'
D4 = 'Sub-Category'

D5 = 'Country/Region'
D6 = 'Region'
D7 = 'State'
D8 = 'City'

D9 = 'Ship Mode'

M1 = ['Quantity','mean']
M2 = ['Discount','sum']
M3 = ['Profit','max']
M4 = ['Sales','count']

row = [D3,D4,D6]
column = [M3]

#column , row = row ,column

lr = []
lc = []

for r in row:
    if type(r) == type(['list']):
        s = str(r[1]+'('+r[0]+')')
        lr.append(s)
    else:
        lr.append(r)

for c in column:
    if type(c) == type(['list']):
        s = str(c[1]+'('+c[0]+')')
        lc.append(s)
    else:
        lc.append(c)

print(row,column)
print(lr,lc)

if len(lr) == 1 and len(lc) == 1:
    c = alt.Chart(df).mark_bar().encode(
    x=lc[-1],
    y=lr[-1]
    ).resolve_scale(x = 'independent',y = 'independent')
    
elif len(lr) == 2 and len(lc) == 1:
    c = alt.Chart(df).mark_bar().encode(
        x=lc[-1],
        y=lr[-1]
    ).facet(row=lr[-2]
    ).resolve_scale(x = 'independent',y = 'independent')

elif len(lr) == 1 and len(lc) == 2:
    c = alt.Chart(df).mark_bar().encode(
        x=lc[-1],
        y=lr[-1]
    ).facet(column=lc[-2]
    ).resolve_scale(x = 'independent',y = 'independent')

elif len(lr) == 2 and len(lc) == 2:
    c = alt.Chart(df).mark_bar().encode(
        x=lc[-1],
        y=lr[-1]
    ).facet(row=lr[-2] , column = lc[-2]
    ).resolve_scale(x = 'independent',y = 'independent')

elif len(lr) == 1 and len(lc) == 3:
    c = alt.Chart(df).mark_bar().encode(
        x=lc[-2],
        y=lr[-1],
        color = lc[-1]
    ).facet(column = lc[-3]
    ).resolve_scale(x = 'independent',y = 'independent')

elif len(lr) == 3 and len(lc) == 1:
    c = alt.Chart(df).mark_bar().encode(
        x=lc[-1],
        y=lr[-2],
        color = lr[-1]
    ).facet(row=lr[-3]
    ).resolve_scale(x = 'independent',y = 'independent')
c

['Category', 'Sub-Category', 'Region'] [['Profit', 'max']]
['Category', 'Sub-Category', 'Region'] ['max(Profit)']


alt.FacetChart(...)

In [70]:

c1 = alt.Chart(df).mark_bar().encode(
    x='Sub-Category',
    y='sum(Quantity)'
).facet(column='Category'
).resolve_scale(x = 'independent',y = 'independent')

c2 = alt.Chart(df).mark_bar().encode(
    x='Sub-Category',
    y='sum(Profit)'
).facet(column='Category'
).resolve_scale(x = 'independent',y = 'independent')

tempmax = df.groupby(['Region','Ship Date month'], as_index=False)['Profit'].mean().max()[2]
tempmin = df.groupby(['Region','Ship Date month'], as_index=False)['Profit'].mean().min()[2]

c3 = alt.Chart(df).mark_bar().encode(
    x='month(Ship Date)',
    y=alt.Y('average(Profit)',scale=alt.Scale(domain=[tempmin,tempmax])),
    tooltip = ['month(Ship Date)','average(Profit)']
).facet(column='Region'
).resolve_scale(x = 'independent',y = 'independent')

#alt.vconcat(c1,c2,c3)
print(tempmin,tempmax)
c1

-12.915894545454544 113.42007450980392


alt.FacetChart(...)